This notebook takes inspiration from this colab notebook: https://colab.research.google.com/drive/1Z5-LqlJxNslHcCzM9jChsNhGi9Y7HSC4. Due to the limitation of the resources we are going to train the code on Google colab. 

<b>Google colab</b> helps us to write and execute python in the browser. We don't have to configure anything as almost everything come pre-configured and most importantly we get free access to GPUs. These GPUs however are shared so, its not necessary that we always get the access but almost all the time apart from once or twice I got the GPUs on colab.

We get a linux box running for a session which typically last for 12 hours if there is a job running in background. In the free tier of the instance we usually get 12.7 GB of system RAM, around 37 GB of disk space and most importantly NVIDIA K80 graphic card. Which is pretty awesome to run the job with higher batch sizes. I tried running the job on my 1070M and 980Ti Chips which are very powerful chips but these chips suffer from a lower RAM size. 

Finally, one more very important thing to remember is to dock the google drive to your colab project. Which is very simple to do. This will help you to save the learning milestones to google drive otherwise the data go away once session ends. Or you can manually download the hdf5 files, which is definitely doesn't sound nice. 

Check if the GPU is enabled.

In [ ]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

<b>device:GPU:0</b> Indicates that the GPU was assigned. If you don't see this, it means that there are no GPU available right now. Try this some other time. 

Now let's download the data and unzip it.

In [ ]:
# Helper function to download data and extract
import os
def get_data_extract():
    if "food-101" in os.listdir():
        print("Dataset already exists")
    else:
        print("Downloading the data...")
        !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
        print("Dataset downloaded!")
        print("Extracting data..")
        !tar xzvf food-101.tar.gz
        print("Extraction done!")

In [ ]:
# listing all the directories in the dataset
import os
os.listdir('food-101/images')

As seen from inceptionV3 notebook that an easy way is to read the data from Train and Test directories. So let's read the train and test data from the json files that come along with the dataset and create new folder separating train and test data.

In [ ]:
# Helper method to split dataset into train and test folders

from shutil import copy
from collections import defaultdict

def prepare_data(filepath, src, dest):
    classes_images = defaultdict(list)
    with open(filepath, 'r') as txt:
        paths = [read.strip() for read in txt.readlines()]
        for p in paths:
            food = p.split('/')
            classes_images[food[0]].append(food[1] + '.jpg')
            
    for food in classes_images.keys():
        print("\nCopying images into ",food)
        if not os.path.exists(os.path.join(dest,food)):
            os.makedirs(os.path.join(dest,food))
        for i in classes_images[food]:
            copy(os.path.join(src,food,i), os.path.join(dest,food,i))

    print("Copying Done!")

In [ ]:
# Prepare train dataset by copying images from food-101/images to food-101/train using the file train.txt
print("Creating train data...")
prepare_data('food-101/meta/train.txt', 'food-101/images', 'food-101/train')

In [ ]:
# Prepare test data by copying images from food-101/images to food-101/test using the file test.txt
print("Creating test data...")
prepare_data('food-101/meta/test.txt', 'food-101/images', 'food-101/test')

Let's see the number of samples to varify the distribution

In [ ]:
# Check how many files are in the test folder
print("Total number of samples in train folder")
!find food-101/train -type d -or -type f -printf '.' | wc -c

print("Total number of samples in test folder")
!find food-101/test -type d -or -type f -printf '.' | wc -c

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np

%matplotlib inline

import os
from os import listdir
from os.path import isfile, join
import shutil
import stat
import collections
from collections import defaultdict

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

import h5py
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model

Remaining code remains the same as the InceptionV3 expect that we store the checkpoints in Google drive. 

In [ ]:
%%time
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input

from keras.models import load_model

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K
import math


K.clear_session()

n_classes = 101

base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(299, 299, 3)))
x = base_model.output
x = AveragePooling2D(pool_size=(8, 8))(x)
x = Dropout(.4)(x)
x = Flatten()(x)
predictions = Dense(n_classes, init='glorot_uniform', W_regularizer=l2(.0005), activation='softmax')(x)

model = Model(input=base_model.input, output=predictions)

opt = SGD(lr=.01, momentum=.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='drive/My Drive/Deeplearning/model/model4.{epoch:02d}-{val_loss:.2f}.hdf5', verbose=1, save_best_only=True)
csv_logger = CSVLogger('drive/My Drive/Deeplearning/log/model4.log')

def schedule(epoch):
    if epoch < 15:
        return .01
    elif epoch < 28:
        return .002
    else:
        return .0004
lr_scheduler = LearningRateScheduler(schedule)

# mixing the old code into GoogleNet
# original sixe of the batch size was 64 but due to the limitation of the GPU memory the batch size is decreased. 
batch_size = 64

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False, # randomly flip images
    zoom_range=[.8, 1],
    channel_shift_range=30,
    fill_mode='reflect')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator()

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
    'food-101/train/',  # this is the target directory
    target_size=(299, 299),  # all images will be resized to 299x299
    batch_size=batch_size,
    seed=42,
    class_mode='categorical')  

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
    'food-101/test/',
    target_size=(299, 299),
    batch_size=batch_size,
    seed=42,
    class_mode='categorical')

model.load_weights('drive/My Drive/Deeplearning/model/model4.08-0.71.hdf5')

model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    validation_steps=25250 // batch_size,
    steps_per_epoch=75750 // batch_size,
    epochs=32,
    callbacks=[lr_scheduler, csv_logger, checkpointer])